In [1]:
import rasterio
import numpy as np
from affine import Affine
from rasterio.windows import from_bounds
import os


In [2]:
fb_path = 'data/humdata/population_nga_2018-10-01.tif'
grid_path = 'data/grid3/NGA - population - v1.2 - mastergrid.tif'

fb_raster = rasterio.open(fb_path)
grid_raster = rasterio.open(grid_path)

In [3]:
def make_bounds(lat, lon, spread):
    return {
        'left': lon - spread,
        'bottom': lat - spread,
        'right': lon + spread,
        'top': lat + spread
    }


In [15]:
# make box around lagos
lat, lon = (6.541456, 3.312719)
spread = 0.1
bounds = make_bounds(lat, lon, spread)
# get anchor pixel in upper left corner of the box to determine translation for new raster
upper_left_fb_lagos = (bounds['left'], bounds['top'])
upper_left_fb_whole = fb_raster.transform * (0,0)
offset = (upper_left_fb_lagos[0]-upper_left_fb_whole[0], upper_left_fb_whole[1] - upper_left_fb_lagos[1])
# get window
left, bottom, right, top = make_bounds(lat, lon, spread).values()
window = from_bounds(left, bottom, right, top, fb_raster.transform, precision=15)
lagos_fb_data = fb_raster.read(1, window=window)
c, f = fb_raster.xy(window.row_off, window.col_off)
#lagos_grid_data = grid_raster.read(1, window=from_bounds(left, bottom, right, top, grid_raster.transform))

In [16]:
transform = Affine(fb_raster.transform.a, fb_raster.transform.b, c, fb_raster.transform.d, fb_raster.transform.e, f)
with rasterio.open(\
    'data/out/fb-lagos-0.5.tif', \
    'w', \
    driver='GTiff', \
    width=lagos_fb_data.shape[1], \
    height=lagos_fb_data.shape[0], \
    count=1, \
    dtype=lagos_fb_data.dtype, \
    crs=fb_raster.crs, \
    transform=transform \
    ) as dst:
    dst.write(lagos_fb_data, indexes=1)


# get some pixels in lagos
#px, py = fb_raster.index(upper_left[0], upper_left[1] op=round, precision=15)
#fb_pixels = np.array([[x,y] for x in np.arange(px, px+10) for y in np.arange(py, py+10)])

In [17]:
fb_lagos_raster = rasterio.open('data/out/fb-lagos-0.5.tif')

In [18]:
# sanity check
#print(fb_lagos_raster.transform * (0,0))
#px, py = fb_raster.index(upper_left_fb_lagos[0], upper_left_fb_lagos[1], op=round, precision=15)
#print(px,py)
print(fb_raster.xy(window.row_off, window.col_off))
print(fb_lagos_raster.xy(0,0))

(3.21285788888889, 6.6413171111111104)
(3.212996777777779, 6.641178222222222)


In [34]:
fb_pixels = np.arange(fb_lagos_raster.height * fb_lagos_raster.width).reshape(fb_lagos_raster.height, fb_lagos_raster.width)
# for each pixel in fb, get coordinates
fb_vxy = np.vectorize(fb_lagos_raster.xy)
fb_xcoords, fb_ycoords = fb_vxy(fb_pixels[:,0], fb_pixels[:,1])

(2.6841273720370373, 13.89210453021605)

In [13]:
# for each pixel in fb, get coordinates
fb_vxy = np.vectorize(fb_raster.xy)
fb_xcoords, fb_ycoords = fb_vxy(fb_pixels[:,0], fb_pixels[:,1])

In [14]:
# get corresponding pixels in grid3
grid_vindex = np.vectorize(grid_raster.index)
grid_pixels = np.vstack(grid_vindex(fb_xcoords, fb_ycoords, op=round, precision=15)).T


grid_pixel_counts = np.unique(grid_pixels, return_counts=True, axis=0)

# TODO
grid_pixel_convolution = np.unique(grid_pixels[fb_data > 0], return_counts=True, axis=0)

In [ ]:
fb_values = fb_raster.read(1, window=Window(px,py,200,200))[fb_pixels[:,0], fb_pixels[:,1]] > 0